In [2]:
from bs4 import BeautifulSoup
import httplib2
import re
import json
import string
h = httplib2.Http(".cache",disable_ssl_certificate_validation=True)
forbidden=set()
class Invalid(RuntimeError):   
    def __init__(self, message):      
        self.message = message
class Forbidden(RuntimeError):   
    def __init__(self, message):      
        self.message = message
def clean(url,TLD=['.org','.net','.edu','.com','.int']):
    if re.match(('^(https://www\.).*(\.(com|org|net|in|edu|int))$'),url.lower()):
        return str(url)
    else:
        url=url.replace('www','').replace('https','').lstrip('.').lstrip(':').lstrip('//')
        url='https://www.'+url
        if (url.lower())[-4:] not in TLD:
            raise Invalid('Invalid URL Top level domain name:clean')
        return str(url)
def search_url(uni,keyword):
    resp,url_html=h.request('http://www.'+uni+'.edu')
    bs_html=BeautifulSoup(url_html)
    url=set()
    if resp['status'] not in ['200','304']:
        forbidden.add(resp['status']+':'+uni)
        raise Forbidden('url forbidden:search_url')
    tag=bs_html.find(action=re.compile('(.*)search(.*)$'))
    if tag == None:
        mid_url='http://www.'+uni+'.edu/search'
        print('tag:'+mid_url)
    else:
        if tag.attrs['action'].find(uni+'.edu') < 0:
            mid_url=('http://www.'+uni+'.edu'+tag.attrs['action'])
        else:
            mid_url=tag.attrs['action']
    url.add(mid_url)
    print(url)
    if 'content-location' in list(h.request(mid_url)[0].keys()):
        mid_url=(h.request(mid_url)[0])['content-location']
        print(mid_url)
    url.add(mid_url)
    mid_bs=BeautifulSoup(h.request(mid_url)[1])
    final_url=mid_url
    for search in mid_bs.findAll('form',{'action':re.compile('^(http)(.*)search(.*)$')}):
        if (search.attrs['action'] not in url) and ((search.attrs['action'].find(uni)) >-1 ):
            final_url=search.attrs['action']
    return (final_url.split('?'))[0]
search_url('harvard','knn')

C:\Users\rishi\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\rishi\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


{'http://www.harvard.edu/searches/'}
https://www.harvard.edu/searches


'https://www.harvard.edu/searches'